### How Delta Lake uses z-order or Data Skipping

In [0]:
# /dbfs/FileStore/Telco_Customer_Churn.csv

In [0]:
raw_path = "/FileStore/Telco_Customer_Churn.csv"
bronze_path = "dbfs:/mnt/delta/telco_churn_bronze"




In [0]:
df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(raw_path))



In [0]:
# Clean known gotcha: TotalCharges sometimes comes as string/blank
from pyspark.sql import functions as F
df = df.withColumn("TotalCharges", F.col("TotalCharges").cast("double"))



In [0]:
(df.write
   .format("delta")
   .mode("overwrite")
   .save(bronze_path))

#### check this path now dbfs:/mnt/delta/telco_churn_bronze

In [0]:
%sql
-- In a SQL cell
CREATE DATABASE IF NOT EXISTS telco;
DROP TABLE IF EXISTS telco.telco_churn;
CREATE TABLE telco.telco_churn
USING DELTA
LOCATION 'dbfs:/mnt/delta/telco_churn_bronze';

#### simple partitioning: if your queries often filter by a few categories (e.g., Contract or InternetService), you may choose small partitioning. Example with a fresh write:

In [0]:
silver_path = "dbfs:/mnt/delta/telco_churn_silver"

(spark.read.format("delta").load(bronze_path)
 .write
 .format("delta")
 .mode("overwrite")
 .partitionBy("Contract")        # 3 partitions; safe and tidy
 .save(silver_path))


spark.sql(
    "DROP TABLE IF EXISTS telco.telco_churn"
)

spark.sql(
    """
    CREATE TABLE telco.telco_churn
    USING DELTA
    LOCATION 'dbfs:/mnt/delta/telco_churn_silver'
    """
)



DataFrame[]

###### Run OPTIMIZE … ZORDER BY (the star of the show)

###### Pick 1–3 frequently-filtered, medium/high-cardinality columns. For this dataset, good starts are:

###### MonthlyCharges (continuous)

###### TotalCharges (continuous)

###### tenure (range filters)

In [0]:
%sql
-- Databricks SQL
OPTIMIZE telco.telco_churn
ZORDER BY (MonthlyCharges, TotalCharges, tenure);


path,metrics
dbfs:/mnt/delta/telco_churn_silver,"List(3, 6, List(96048, 201940, 134102.66666666666, 3, 402308), List(45024, 154319, 80654.33333333333, 6, 483926), 4, List(minCubeSize(107374182400), List(0, 0), List(7, 566962), 0, List(6, 483926), 3, null), null, 0, 1, 7, 1, false, 0, 0, 1756183362160, 1756183372710, 4, 3, null, List(0, 0), null, 23, 23, 1559, 0, null)"


##### Tip: Don’t include very low-cardinality columns (e.g., Churn) and avoid unique IDs (customerID) unless you truly query by equality on them a lot.

#### Verify it worked

In [0]:
%sql
-- See the compaction & Z-Order job details
OPTIMIZE telco.telco_churn ZORDER BY (MonthlyCharges, TotalCharges, tenure);

path,metrics
dbfs:/mnt/delta/telco_churn_silver,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 1, List(minCubeSize(107374182400), List(3, 402308), List(1, 83036), 1, List(0, 0), 0, null), null, 0, 0, 4, 4, false, 0, 0, 1756183439579, 1756183442598, 4, 0, null, List(0, 0), null, 23, 23, 0, 0, null)"


In [0]:
%sql
-- Check table/file details
DESCRIBE DETAIL telco.telco_churn;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,37583c3a-97d5-4838-b323-a81015ab4a8e,spark_catalog.telco.telco_churn,null,dbfs:/mnt/delta/telco_churn_silver,2025-08-26T04:38:27.375Z,2025-08-26T04:42:52Z,List(Contract),List(),4,485344,Map(delta.enableDeletionVectors -> true),3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
-- History shows OPTIMIZE runs
DESCRIBE HISTORY telco.telco_churn;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-08-26T04:42:52Z,2406467808806708,akankshasaxena390@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [""MonthlyCharges"",""TotalCharges"",""tenure""], batchId -> 0)",null,List(406205978710828),0826-042339-ngz4bmw2,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 483926, p25FileSize -> 96048, numDeletionVectorsRemoved -> 0, minFileSize -> 96048, numAddedFiles -> 3, maxFileSize -> 201940, p75FileSize -> 201940, p50FileSize -> 104320, numAddedBytes -> 402308)",null,Databricks-Runtime/16.4.x-cpu-ml-scala2.12
1,2025-08-26T04:39:09Z,2406467808806708,akankshasaxena390@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [""Contract""])",null,List(406205978710828),0826-042339-ngz4bmw2,0,WriteSerializable,false,"Map(numFiles -> 7, numRemovedFiles -> 7, numRemovedBytes -> 566962, numOutputRows -> 10000, numOutputBytes -> 566962)",null,Databricks-Runtime/16.4.x-cpu-ml-scala2.12
0,2025-08-26T04:38:31Z,2406467808806708,akankshasaxena390@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [""Contract""])",null,List(406205978710828),0826-042339-ngz4bmw2,null,WriteSerializable,false,"Map(numFiles -> 7, numRemovedFiles -> 0, numRemovedBytes -> 0, numOutputRows -> 10000, numOutputBytes -> 566962)",null,Databricks-Runtime/16.4.x-cpu-ml-scala2.12


In [0]:
%sql
-- Quick query you care about (should scan fewer files after Z-Order)
EXPLAIN COST
SELECT *
FROM telco.telco_churn
WHERE MonthlyCharges BETWEEN 60 AND 80
  AND tenure >= 24;

plan
"== Optimized Logical Plan == Filter ((isnotnull(MonthlyCharges#7089) AND isnotnull(tenure#7076)) AND (((MonthlyCharges#7089 >= 60.0) AND (MonthlyCharges#7089 <= 80.0)) AND (tenure#7076 >= 24))), Statistics(sizeInBytes=392.9 KiB, ColumnStat: N/A) +- Relation spark_catalog.telco.telco_churn[customerID#7071,gender#7072,SeniorCitizen#7073,Partner#7074,Dependents#7075,tenure#7076,PhoneService#7077,MultipleLines#7078,InternetService#7079,OnlineSecurity#7080,OnlineBackup#7081,DeviceProtection#7082,TechSupport#7083,StreamingTV#7084,StreamingMovies#7085,Contract#7086,PaperlessBilling#7087,PaymentMethod#7088,MonthlyCharges#7089,TotalCharges#7090,Churn#7091,Question#7092,Answer#7093] parquet, Statistics(sizeInBytes=392.9 KiB, ColumnStat: N/A) == Physical Plan == *(1) Project [customerID#7071, gender#7072, SeniorCitizen#7073, Partner#7074, Dependents#7075, tenure#7076, PhoneService#7077, MultipleLines#7078, InternetService#7079, OnlineSecurity#7080, OnlineBackup#7081, DeviceProtection#7082, TechSupport#7083, StreamingTV#7084, StreamingMovies#7085, Contract#7086, PaperlessBilling#7087, PaymentMethod#7088, MonthlyCharges#7089, TotalCharges#7090, Churn#7091, Question#7092, Answer#7093] +- *(1) Filter (((((if (isnotnull(_databricks_internal_edge_computed_column_skip_row#7234)) (_databricks_internal_edge_computed_column_skip_row#7234 = false) else isnotnull(raise_error(DELTA_SKIP_ROW_COLUMN_NOT_FILLED, map(keys: [], values: []), NullType)) AND isnotnull(MonthlyCharges#7089)) AND isnotnull(tenure#7076)) AND (MonthlyCharges#7089 >= 60.0)) AND (MonthlyCharges#7089 <= 80.0)) AND (tenure#7076 >= 24)) +- *(1) ColumnarToRow +- FileScan parquet spark_catalog.telco.telco_churn[customerID#7071,gender#7072,SeniorCitizen#7073,Partner#7074,Dependents#7075,tenure#7076,PhoneService#7077,MultipleLines#7078,InternetService#7079,OnlineSecurity#7080,OnlineBackup#7081,DeviceProtection#7082,TechSupport#7083,StreamingTV#7084,StreamingMovies#7085,PaperlessBilling#7087,PaymentMethod#7088,MonthlyCharges#7089,TotalCharges#7090,Churn#7091,Question#7092,Answer#7093,_databricks_internal_edge_computed_column_skip_row#7234,Contract#7086] Batched: true, DataFilters: [isnotnull(MonthlyCharges#7089), isnotnull(tenure#7076), (MonthlyCharges#7089 >= 60.0), (MonthlyC..., Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[dbfs:/mnt/delta/telco_churn_silver], PartitionFilters: [], PushedFilters: [IsNotNull(MonthlyCharges), IsNotNull(tenure), GreaterThanOrEqual(MonthlyCharges,60.0), LessThanO..., ReadSchema: struct COMPUTE STATISTICS FOR ALL COLUMNS"


#### you’re looking at the Spark physical plan after Z-ORDER (plus Delta’s skipping logic). Let me break this down for you step by step:

#### What happens without Z-ORDER?

Normally, a Delta/Parquet table is just a bunch of files (say, 500 small files).
If you filter on:

WHERE MonthlyCharges BETWEEN 60 AND 80
  AND tenure >= 24


Spark doesn’t know which files have rows in that range — so it has to scan all files, checking row by row. That’s expensive.

#### What Z-ORDER does

When you run:

OPTIMIZE telco.telco_churn
ZORDER BY (MonthlyCharges, TotalCharges, tenure)


Databricks physically rewrites the files.
It uses a space-filling curve (Z-curve) to reorder rows so that values that are close together for those columns end up stored together in the same set of files/pages.

Rows with MonthlyCharges 60–80 and tenure >= 24 are now clustered into fewer files.

Other ranges are pushed into separate files.

#### How Delta Lake uses Data Skipping

Delta automatically maintains min/max statistics per file (and per data block, if enabled).

Example (simplified file stats after Z-ORDER):

File A: MonthlyCharges [10–59]
File B: MonthlyCharges [60–80], tenure [20–60]
File C: MonthlyCharges [81–120]


When your query runs, Spark/Delta sees:

File A → skip (outside filter range)

File B → read (overlaps range)

File C → skip

So instead of scanning 500 files, Spark might scan only 20. That’s why scan time drops.

#### Proof in your plan

Look at your plan snippet:

if (isnotnull(_databricks_internal_edge_computed_column_skip_row#7234)) ...


That _databricks_internal_edge_computed_column_skip_row is the data skipping predicate column that Delta injects after OPTIMIZE ZORDER.

#### It marks rows/files that should be skipped before Spark even reads them.

Also note:

PreparedDeltaFileIndex(1 paths) ...
PartitionFilters: []
PushedFilters: [IsNotNull(MonthlyCharges), IsNotNull(tenure), GreaterThanOrEqual(MonthlyCharges,60.0), LessThanOrEqual(MonthlyCharges,80.0), GreaterThanOrEqual(tenure,24)]


This means your filter was pushed down all the way to the file scan level. Files outside those ranges are skipped.

#### Why statistics are still “missing”


That’s because Spark doesn’t yet have column-level histograms for the optimizer. It’s just using Delta’s file-level min/max stats.
If you run:

In [0]:
%sql
ANALYZE TABLE telco.telco_churn COMPUTE STATISTICS FOR ALL COLUMNS;


#### In short:

Z-ORDER clusters related rows into fewer files.

Delta’s data skipping uses file-level min/max stats to skip irrelevant files.

Your query scans far fewer files → less I/O → lower latency.

### How to pick Z-ORDER columns (quick rules)

👍 Frequently used in WHERE or JOIN keys.

👍 Medium/high cardinality (dozens to many distinct values).

👍 Range/equality filters (dates, amounts, numeric measures).

👎 Very low cardinality (TRUE/FALSE flags, 2–5 categories).

👎 Mostly NULLs.

👎 Unique IDs unless equality filters dominate.